In [ ]:
# EDA of the CrisisBench dataset

In [ ]:
import os

import pandas as pd

In [ ]:
data_path = "../data/data/all_data_en"

informativeness_dev = pd.read_csv(os.path.join(data_path, "crisis_consolidated_humanitarian_filtered_lang_en_dev.tsv"), sep="\t")